<div>
    <center>
    <h1> Spring Pendulum </h1>
    <br>
    by <a href="http://github.com/ComputoCienciasUniandes"> ComputoCienciasUniandes </a>
    <br>
    <a href="http://github.com/jsbarbosa"> Juan Barbosa </a>,
    <a href="http://github.com/jsbarbosa"> Luisa Rodriguez </a>
    </center>
</div>
<hr style="height:5px">

# Librerías

In [1]:
from numpy import sin, cos
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation

%matplotlib

Using matplotlib backend: TkAgg


# Sistema

<img src="spring.jpg">

El sistema que vamos a analizar se muestra en la Figura anterior, donde $l$ es la longitud natural del resorte y $r$ lo que se enlonga.

El Lagrangiano del sistema es:

\begin{equation}
L = T-V = \left(\frac{1}{2}m(\dot r^2 + (l+r)^2\dot\theta^2)\right) - \left(-mg\cos(l+r)\cos\theta + \frac{1}{2}kr^2\right)
\end{equation}

De donde se deducen las siguientes ecuaciones de movimiento:
$$
    \dfrac{d}{dt}\dfrac{\partial L}{\partial \dot{q}} - \dfrac{\partial L}{\partial q} = 0
$$

$$
    \ddot r = (l+r)\dot\theta^2 + g\cos\theta - \omega^2 r \qquad \text{donde $\omega^2 = k/m$} 
$$

$$
    \ddot \theta = - \dfrac{1}{l+r}\left(2\dot{r}\dot{\theta} + g\sin\theta\right)
$$


# Constantes

Definimos las constantes a usar.

In [2]:
G = 9.8  # acceleration due to gravity, in m/s^2
K = 100 # spring constant
L = 1.0  # length of pendulum 1 in m
M = 1.0  # mass of pendulum 1 in kg
W2 = K/M

# Condiciones iniciales

In [3]:
r0 = 0.5*L
vr0 = 0
theta0 = 45
vtheta0 = 0

# initial state
state = [r0, vr0, np.deg2rad(theta0), np.deg2rad(vtheta0)]

# Soluciones

In [4]:
def derivs(state, t):
    dydx = np.zeros_like(state)
    r_, vr_, theta_, vtheta_ = state
    dydx[0] = vr_
    dydx[1] = (L + r_)*vtheta_**2 + G*np.cos(theta_) - W2*r_
    dydx[2] = vtheta_
    dydx[3] = -(2*vr_*vtheta_ + G*np.sin(theta_))/(r_ + L)
    return dydx

dt = 0.025
t = np.arange(0.0, 20.0, dt)

# integrate your ODE using scipy.integrate.
y = integrate.odeint(derivs, state, t)
r = L + y[:, 0] 
x1 = r*np.sin(y[:, 2])
y1 = -r*np.cos(y[:, 2])

fig = plt.figure()
rmax = np.ceil(max(abs(r)))
ax = fig.add_subplot(111, autoscale_on = False, xlim = (-rmax, rmax), ylim = (-rmax, rmax))
ax.grid()

line, = ax.plot([], [], 'o-', lw = 2)
story, = ax.plot([], [], alpha = 0.5)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

ax.set_xlabel("$x$ (m)")
ax.set_ylabel("$y$ (m)")

def init():
    line.set_data([], [])
    story.set_data([], [])
    time_text.set_text('')
    return line, time_text

def animate(i):
    thisx = [0, x1[i]]
    thisy = [0, y1[i]]
    story.set_data(x1[:i], y1[:i])

    line.set_data(thisx, thisy)
    time_text.set_text(time_template % (i*dt))
    return line, story, time_text

ani = animation.FuncAnimation(fig, animate, np.arange(1, len(y)),
                              interval = 25, blit = True, init_func=init)

# ani.save("springPendulum.gif", writer = "imagemagick", dpi = 70)
plt.show()